In [5]:
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import math
import time
import datetime
import cv2
import h5py
from sklearn.model_selection import train_test_split
from keras.models import Sequential, Model
from keras.applications import *
from keras.layers import Dropout, Flatten, Dense, Input, GlobalAveragePooling2D
from keras.layers.normalization import *
from keras.preprocessing.image import *

from keras.models import load_model
from keras.preprocessing.image import *

def get_img(path, image_shape):
    img = cv2.imread(path, 0)
    resized = cv2.resize(img, image_shape)
    return resized

def KNN(input_data, predictions, N=5, coef = [0.05, 0.1, 0.15, 0.2, 0.5]):
    new_predictions_list = []
    for i in tqdm(range(len(input_data))):
        distance = np.square(np.subtract(input_data, input_data[i]))
        distance = np.sqrt(np.sum(distance, axis=(1, 2)))
        N_min_index = np.argsort(distance)[::-1][-N:]
        
        pred = [ predictions[N_min_index[j]] * coef[j] for j in range(N)]
        pred = np.sum(pred, axis=0)
        new_predictions_list.append(pred)
                
    return np.array(new_predictions_list)

def make_predictions(MODEL, image_size, batch_size, preprocessing=None):
    gen = ImageDataGenerator(preprocessing_function=preprocessing)
    path_test_data = 'dataset/to_prediction'
    test_generator = gen.flow_from_directory(path_test_data,  image_size, shuffle=False, 
                                             batch_size=batch_size, class_mode='categorical')
    y_predictions = MODEL.predict_generator(test_generator,  steps=test_generator.samples//batch_size+1,  verbose=1)
    # y_predictions = y_predictions.clip(min=0.005, max=0.995)
    
    test_id = list()
    for i, file_name in enumerate(test_generator.filenames):
        flbase = os.path.basename(file_name)
        test_id.append(flbase)       
    
    return y_predictions, test_id

def create_submission(predictions, test_id, tag = ""):
    result1 = pd.DataFrame(predictions, columns=['c0', 'c1', 'c2', 'c3',
                                                 'c4', 'c5', 'c6', 'c7',
                                                 'c8', 'c9'])
    result1.loc[:, 'img'] = pd.Series(test_id, index=result1.index)
    now = datetime.datetime.now()
    if not os.path.isdir('subm'):
        os.mkdir('subm')
    suffix = str(now.strftime("%Y-%m-%d-%H-%M"))
    sub_file = os.path.join('subm', 'submission_' + tag + '.csv')
    result1.to_csv(sub_file, index=False)

In [2]:
model = load_model('models/model_VGG16_BN_15_SGD_10epoche.h5')

In [6]:
y_predictions, test_id = make_predictions(model, (224, 224), 128)

Found 79726 images belonging to 1 classes.
 34/623 [>.............................] - ETA: 10508s

KeyboardInterrupt: 

In [ ]:
new_predictions = y_predictions.clip(min=0.005, max=0.995)

In [ ]:
create_submission(y_predictions, test_id, tag="VGG16_SGD_20e")